# Project 04 Scratch Notebook

In [14]:
import twint
import numpy as np
import pandas as pd
import nest_asyncio
nest_asyncio.apply()

In [3]:
import functools

In [65]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import NMF

In [501]:
# Make better use of Jupyter Notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 400)
pd.options.display.float_format = '{:,.10f}'.format

## Pull in sample data from Twitter using Twint

In [ ]:
#configuration
config = twint.Config()
config.Search = "@VAVetBenefits"
config.Lang = "en"
config.Limit = 1000
#config.Since = '2020–12–18'
#config.Until = time_until
config.Store_csv = True
config.Output = "custom_out_vba_sample.csv"

#running search
twint.run.Search(config)

In [ ]:
#configuration
config = twint.Config()
config.Search = "@DeptVetAffairs"
config.Lang = "en"
config.Limit = 1000
#config.Since = '2020–12–18'
#config.Until = time_until
config.Store_csv = True
config.Output = "custom_out_VA_sample.csv"

#running search
twint.run.Search(config)

In [ ]:
#configuration
config = twint.Config()
config.Search = "@VeteransHealth"
config.Lang = "en"
config.Limit = 1000
#config.Since = '2020–12–18'
#config.Until = time_until
config.Store_csv = True
config.Output = "custom_out_VHA_sample2.csv"

#running search
vha_sample = twint.run.Search(config)

In [146]:
vba_sample = pd.read_csv('/Users/arcarter/Git_Repos/project_04/custom_out_vba_sample.csv')  

In [147]:
vha_sample = pd.read_csv('/Users/arcarter/Git_Repos/project_04/custom_out_VHA_sample2.csv')

In [148]:
va_sample = pd.read_csv('/Users/arcarter/Git_Repos/project_04/custom_out_VA_sample.csv')

In [9]:
va_sample.shape

(1000, 36)

## Create new columns for VBA vs VHA vs VA Tweets 
(Allowing multiple to be selected for same tweet)

In [150]:
vba_sample['VBA Tweet'] = 'Yes'

In [151]:
vba_sample.shape

(1053, 37)

In [152]:
vha_sample['VHA Tweet'] = 'Yes'

In [153]:
vha_sample.shape

(1072, 37)

In [154]:
va_sample['VA Tweet'] = 'Yes'

In [155]:
vba_cols = list(vba_sample.columns) 
vha_cols = list(vha_sample.columns) 
va_cols = list(va_sample.columns) 

In [157]:
no_tweet_vba = vba_cols[:-1]

In [158]:
no_tweet_vha = vha_cols[:-1]

In [159]:
no_tweet_va = va_cols[:-1]

## Merge all versions

In [161]:
merged_all = pd.merge(left=vba_sample, right=va_sample, how='outer', left_on=no_tweet_vba, right_on=no_tweet_vha)
merged_all.sample(10);
    

In [162]:
merged_all_w_vha = pd.merge(left=merged_all, right=vha_sample, how='outer', left_on=no_tweet_vba, right_on=no_tweet_vha)
merged_all_w_vha.sample(10);

In [ ]:
merged_all_w_vha_copy = merged_all_w_vha.copy()

In [163]:
merged_all_w_vha.loc[merged_all_w_vha['VBA Tweet'] == 'Yes', 'VBA Tweet'] = 1
merged_all_w_vha.loc[merged_all_w_vha['VA Tweet'] == 'Yes', 'VA Tweet'] = 1
merged_all_w_vha.loc[merged_all_w_vha['VHA Tweet'] == 'Yes', 'VHA Tweet'] = 1

## Send version to CSV (and Pickle)

In [169]:
merged_all_w_vha.to_csv(index=False,path_or_buf='/Users/arcarter/Git_Repos/project_04/merged_all_w_vha.csv')

In [170]:
merged_all_w_vha.head();

In [10]:
import pickle

In [171]:
pickle.dump(merged_all_w_vha, open("merged_all_w_vha.p", "wb" ) )

In [172]:
merged_all_w_vha_pickle = pickle.load( open( "merged_all_w_vha.p", "rb" ) )

## Create stand alone list of just tweets
With a few additional columns including time and if it's VBA, VA and/or VHA Focused

In [173]:
df_tweets_2 = pd.DataFrame(merged_all_w_vha[["tweet", "id","date","time","VBA Tweet","VA Tweet","VHA Tweet"]])

In [17]:
type(df_tweets_2)

pandas.core.frame.DataFrame

In [18]:
df_tweets_2.shape

(2985, 5)

In [242]:
df_tweets_2.head()

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet
0,It's time for @SMC_CVSO @sanmateoco #veterans to rise up and demand services to be a priority! #homelessness in our county needs to end...@SMCHSA lets put @VeteransHealth and @VAVetBenefits to work and truly end #veteran homelessness in @sanmateoco,1361892469500121089,2021-02-16,23:17:22,1,NaN,1
1,"There should be another bill that temporarily raises @SocialSecurity and @VAVetBenefits to $2,500 for 6-12 months. New @HUDgov vouchers for apartments, rent rooms, &amp; motel rooms to partially deal w/ housing crisis. Expanded SNAP for unemployed. Extend Lifeline to SS retire. bene.",1361886795932262407,2021-02-16,22:54:49,1,NaN,NaN
2,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,2021-02-16,21:33:57,1,NaN,NaN
3,"RT @VAVetBenefits : (2/2) In 2014, Howard became the first female four-star Admiral &amp; the first to command operational forces, making her the highest ranking woman &amp; African-American in Navy history. After a remarkable career, Adm. Howard retired in 2017. Thank you for your …",1361854662488555525,2021-02-16,20:47:08,1,NaN,NaN
4,"RT @VAVetBenefits : #VBATribute | Trailblazer Michelle Howard set many firsts during her 35-year career with @USNavy. A Veteran of the Gulf War, she served in Operation Desert Shield and Desert Storm &amp; in 1999, became the first African-American woman to … https://t.co/JPSmR9ryRQ",1361854659409965057,2021-02-16,20:47:07,1,NaN,NaN


## Pre-Process Tweet Data

In [22]:
import re
import string

#### Create copies of the base dataframe

In [444]:
df_tweets_final = df_tweets_2.copy()

In [436]:
df_tweets_2[df_tweets_2['tweet'].str.contains("vaccine")]

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet
30,"@LeoShane @TheVeteransFirm @DeptVetAffairs #disabled #veteran that for the last 34 years have been tormented by the @DeptVetAffairs @VAVetBenefits @HouseVetAffairs @MemphisVAMC There has been much failure, CUE and oversight on my claims. I’ve not gotten the vaccine yet, but I do have a thoracic aortic aneurysms. 11/20",1361678095183454211,2021-02-16,09:05:31,1,1,NaN
104,@ABC7NY Yesterday I went with fellow veteran so he could get Covid vaccine at @NorthportVAMC never was so impressed in the system they created to get 1st and 2nd shots. No drama @NYGovCuomo @NYCMayor @DeptVetAffairs @VAVetBenefits @VeteransHealth,1360577851137929216,2021-02-13,08:13:32,1,1,1
124,Please @VAVetBenefits @DeptVetAffairs allow vet spouses to receive the vaccine at the same time the sponsor does. Multiple trips to 2 different places is inefficient and takes more time. @PressSec @VP @POTUS Consolidate and crunch the virus!!,1360307023116664832,2021-02-12,14:17:22,1,NaN,NaN
178,RT @VAVetBenefits : RT @DeptVetAffairs: America's oldest WWII Veteran received his COVID-19 vaccine at the Veterans Affairs Medical Center in New Orleans on Monday https://t.co/H5Er5WCgoi via @wdsu,1359484056325545987,2021-02-10,07:47:11,1,NaN,NaN
197,"RT @VAVetBenefits : RT @DeptVetAffairs: Marine Veteran and World War II Navajo Code Talker Thomas Begay survived Iwo Jima. Recently, he received his COVID-19 vaccine at @VANewMexico to protect himself and loved ones from the virus. #HonoringVets #COVIDV… https://t.co/GLIW0nbbF8",1359197133484744705,2021-02-09,12:47:03,1,NaN,NaN
352,"? for @VAVetBenefits and @hawleymo. Why doesn't VA have a site where vets not part of the system can sign up for vaccine? KCMO vet, 73 yo, cancer, 3 yr overseas and the VA won't help me!",1355971247448989698,2021-01-31,15:08:32,1,NaN,NaN
490,@ConcernedVets @DeptVetAffairs @HouseVetAffairs @VAVetBenefits @SpecialReport @RepCharlieCrist @SenRickScott @marcorubio @TB_Times @realannapaulina Pls clarify those Florida news reports on Vets’ income and Covid vaccine access. Thank you.,1352587376791588868,2021-01-22,07:02:15,1,NaN,NaN
563,@DeptVetAffairs @VAVetBenefits ... WHY is the West Palm Beach VA saying they won’t give you a Covid vaccine if you make above $34K per year ?? This vaccine is FREE ... paid for by my taxes ... you are killing older veterans @CBS12 @WPBF25News @WPTV @marcorubio @JoeBiden,1351537149796114433,2021-01-19,09:29:01,1,NaN,NaN
611,What's the holdup on covid-19 vaccines? @tampava @VAVetBenefits,1350609317586743299,2021-01-16,20:02:09,1,NaN,NaN
698,At the VA getting my free health care due to some years in the sand. Should I opt for the Covid-19 vaccine or save it for an elderly veteran? @VAVetBenefits #PTSD #COVIDー19 @VA_PTSD_Info,1349793727066365957,2021-01-14,14:01:17,1,NaN,NaN


In [230]:
df_tweets_final.shape

(2882, 8)

In [445]:
df_tweets_final.head(2)

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet
0,It's time for @SMC_CVSO @sanmateoco #veterans to rise up and demand services to be a priority! #homelessness in our county needs to end...@SMCHSA lets put @VeteransHealth and @VAVetBenefits to work and truly end #veteran homelessness in @sanmateoco,1361892469500121089,2021-02-16,23:17:22,1,NaN,1
1,"There should be another bill that temporarily raises @SocialSecurity and @VAVetBenefits to $2,500 for 6-12 months. New @HUDgov vouchers for apartments, rent rooms, &amp; motel rooms to partially deal w/ housing crisis. Expanded SNAP for unemployed. Extend Lifeline to SS retire. bene.",1361886795932262407,2021-02-16,22:54:49,1,NaN,NaN


In [ ]:
247fubar

#### Create pre-processing helper function

In [492]:
def update_base_table_for_NMF(df):
    
    # Pre-process data
    df.tweet = df.tweet.map(no_rt_marker)
    df.drop_duplicates(subset='tweet',inplace=True)
    df['tweet'] = df.tweet.map(no_at_symbol)    # replace with a ' ', vs the rest of the replaces which will replace with ''
    df['tweet'] = df.tweet.map(no_lb_symbol)    # replace with a ' ', vs the rest of the replaces which will replace with ''
    df['tweet'] = df.tweet.map(punc_lower)
    df['tweet'] = df.tweet.map(no_website)
    df['tweet'] = df.tweet.map(SSA).map(SSA2).map(VBA).map(VBA2).map(VHA).map(VHA2).map(veteran_words).map(va_words).map(va_words2)
    df['tweet'] = df.tweet.map(black_veterans_words)
    df['tweet'] = df.tweet.map(covid_words) 
    df['tweet'] = df.tweet.map(vaccine_words) 
    df['tweet'] = df.tweet.map(alphanumeric)
    df['tweet'] = df.tweet.apply(remove_emoji)
        
    return df.head()

In [475]:
def update_base_table_for_NMF_test(df,lookup_word):
    
    # Pre-process data
    df.tweet = df.tweet.map(no_rt_marker)
    df.drop_duplicates(subset='tweet',inplace=True)
    #df['tweet'] = df.tweet.map(no_at_symbol)    # replace with a ' ', vs the rest of the replaces which will replace with ''
    #df['tweet'] = df.tweet.map(no_lb_symbol)    # replace with a ' ', vs the rest of the replaces which will replace with ''
    #df['tweet'] = df.tweet.map(punc_lower)
    #df['tweet'] = df.tweet.map(no_website)
    #df['tweet'] = df.tweet.map(SSA).map(SSA2).map(VBA).map(VBA2).map(VHA).map(VHA2).map(veteran_words).map(va_words).map(va_words2)
    #df['tweet'] = df.tweet.map(black_veterans_words)
    #df['tweet'] = df.tweet.map(covid_words) 
    #df['tweet'] = df.tweet.map(vaccine_words) 
    df['tweet'] = df.tweet.map(alphanumeric)
    #df['tweet'] = df.tweet.apply(remove_emoji)
        
    return df[df['tweet'].str.contains("{}".format(lookup_word))].head(20)

In [465]:
df_tweets_final.head(2)

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet
0,It's time for @SMC_CVSO @sanmateoco #veterans to rise up and demand services to be a priority! #homelessness in our county needs to end...@SMCHSA lets put @VeteransHealth and @VAVetBenefits to work and truly end #veteran homelessness in @sanmateoco,1361892469500121089,2021-02-16,23:17:22,1,NaN,1
1,"There should be another bill that temporarily raises @SocialSecurity and @VAVetBenefits to $2,500 for 6-12 months. New @HUDgov vouchers for apartments, rent rooms, &amp; motel rooms to partially deal w/ housing crisis. Expanded SNAP for unemployed. Extend Lifeline to SS retire. bene.",1361886795932262407,2021-02-16,22:54:49,1,NaN,NaN


In [493]:
df_tweets_final = df_tweets_2.copy()

In [130]:
#df_tweets_8_c.drop(df_tweets_8_c.iloc[:, 5:14], inplace = True, axis = 1)

In [512]:
df_tweets_final[df_tweets_final['tweet'].str.contains("jane")].head(20)

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet,topic_count_10,topic_count_15
1520,the vaccine rollout at the department of veterans affairs with dr jane kim chief consultant for preventative medicine at department_of_va,1361318094455545857,2021-02-15,09:15:00,NaN,1,NaN,6,8
1654,department_of_va has vaccinated over people with the coronavirus_pandemic vaccine dr jane kim chief consultant for preventative medicine at department_of_va,1360975838578044933,2021-02-14,10:35:00,NaN,1,NaN,6,8
1869,tmrw dr jane kim from department_of_va on vaccine rollout alex cohen from usgsa on securing digital workers and karen evans and mark forman former egov administrators at ombpress on prioritizing it modernization in government sun at am on wjla news,1360634844133412866,2021-02-13,12:00:00,NaN,1,NaN,6,8
2383,Veterans_Health_Admin department_of_va why are my fellow veterans vietnam era in maine still waiting to be vaccinated govjanetmills mainedhhs mainevets mikeshepherdme pressherald kjonline americanlegion wreathsacross vfwhq rayrichardsonjr,1359284053766246402,2021-02-09,18:32:27,NaN,NaN,1,6,7
2818,pharmacy shines amid coronavirus_pandemic darkness coping tips shared during ashp accp annual meetings ashpofficial accp nychealthsystem janellepoyant tuftsmedicalctr urmed ligneripharmd cleclinicmd Veterans_Health_Admin,1355976696902656000,2021-01-31,15:30:12,NaN,NaN,1,6,8


In [495]:
update_base_table_for_NMF(df_tweets_final)

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet
0,its time for smccvso sanmateoco veterans to rise up and demand services to be a priority homelessness in our county needs to end smchsa lets put Veterans_Health_Admin and Veterans_Benefits_Admin to work and truly end veteran homelessness in sanmateoco,1361892469500121089,2021-02-16,23:17:22,1,NaN,1
1,there should be another bill that temporarily raises Social_Security and Veterans_Benefits_Admin to for months new hudgov vouchers for apartments rent rooms amp motel rooms to partially deal w housing crisis expanded snap for unemployed extend lifeline to Social_Security retire bene,1361886795932262407,2021-02-16,22:54:49,1,NaN,NaN
2,Veterans_Benefits_Admin mr secretary mcdonough good afternoon as you can see from my attached documentation i have been fighting this issue of changing my discharge due to systemic racism for over years,1361866444410155013,2021-02-16,21:33:57,1,NaN,NaN
3,Veterans_Benefits_Admin in howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in thank you for your …,1361854662488555525,2021-02-16,20:47:08,1,NaN,NaN
4,Veterans_Benefits_Admin vbatribute trailblazer michelle howard set many firsts during her career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in became the first africanamerican woman to …,1361854659409965057,2021-02-16,20:47:07,1,NaN,NaN


In [527]:
df_tweets_2[df_tweets_2['tweet'].str.contains("Black")]

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet
192,"The Harlem Hellfighters, aka the 369th Infantry Regiment, spent 191 days on the front, more than any other American unit. The Hellfighters were the first Americans to get the French Croix de Guerre &amp; were the most highly decorated Black combat regiment of WWI. #BlackHistoryMonth https://t.co/lgWwwbA3s2",1359221312451973128,2021-02-09,14:23:08,1,NaN,NaN
279,RT @VAVetBenefits : RT @DeptVetAffairs: AARP celebrates Black Veterans who served https://t.co/YbHapYW2Xo via #VAntagePoint,1357717486934888453,2021-02-05,10:47:28,1,NaN,NaN
319,RT @VAVetBenefits : This month we reflect on and celebrate the many contributions of Black Americans in history. There are over 18M U.S. Veterans and over 2M of them are Black. We honor all Black service members and Veterans who've faithfully served this country. Thank you f…,1356630289477480448,2021-02-02,10:47:20,1,NaN,NaN
321,This month we reflect on and celebrate the many contributions of Black Americans in history. There are over 18M U.S. Veterans and over 2M of them are Black. We honor all Black service members and Veterans who've faithfully served this country. Thank you for your service!,1356629966360870914,2021-02-02,10:46:03,1,NaN,NaN
398,@BlackVetProject @POTUS @VP @DeptVetAffairs @VeteransHealth @VAVetBenefits @NABMW @blackvets @NabvetsInc Diverse Leadership is crucial &amp; critical for the health &amp; well-being of all Veterans who seek services &amp; treatment at the Department of Veterans Affairs ... Cultural competency and commitment to justice within all segments of VA @DeptVetAffairs @VeteransHealth @VAVetBenefits ...,1354735875926728707,2021-01-28,05:19:37,1,NaN,NaN
401,@BlackVetProject @KrisGoldsmith85 @POTUS @VP @DeptVetAffairs @VeteransHealth @VAVetBenefits @NABMW @blackvets @NabvetsInc Good!,1354549151934140416,2021-01-27,16:57:38,1,NaN,NaN
403,"Today, the Black Veteran Empowerment Council, of which @BlackVetProject is a member — authored an open letter to @POTUS &amp; @VP urging the appointment of diverse leadership at the @DeptVetAffairs — @VeteransHealth &amp; @VAVetBenefits. Cc: @NABMW @blackvets @NabvetsInc https://t.co/05NifSKOBq",1354531274883555331,2021-01-27,15:46:36,1,NaN,NaN
665,"@VAVetBenefits The ""Emancipation Proclamation"" announced the acceptance of Black men into the Union Army and the Navy. By the end of the war, almost 200,000 black soldiers and sailors enrolled in the fight for the Union and freedom.",1350030147186008065,2021-01-15,05:40:44,1,NaN,NaN
688,@MarshaBlackburn PLAYBOY PLAYGIRL PENTHOUSE HUSTLER JUGGS Washington DC They cannot handle The TRUTH @congressdotgov @USSupremeCourt @SupremeCourtGA USA AMERICA USA 🇺🇸 GEORGIA DEAD VOTES PORN @VAVetBenefits @UTKnoxville RAPE @NCAA,1349886503363768320,2021-01-14,20:09:56,1,NaN,NaN
793,RT @VAVetBenefits : @JakeBlackSCV .@JakeBlackSCV please respond to the DM I just sent. We'll find a contact for you who'll reach out about your appointment.,1348733272403148801,2021-01-11,15:47:25,1,NaN,NaN


In [525]:
df_tweets_final[df_tweets_final['tweet'].str.contains("vaccine")]

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet,topic_count_10,topic_count_15
30,leoshane theveteransfirm department_of_va disabled veteran that for the last years have been tormented by the department_of_va Veterans_Benefits_Admin housevetaffairs memphisvamc there has been much failure cue and oversight on my claims i’ve not gotten the vaccine yet but i do have a thoracic aortic aneurysms,1361678095183454211,2021-02-16,09:05:31,1,1,NaN,3,6
58,Veterans_Benefits_Admin vaveteranssvcs Veterans_Health_Admin maziehirono brianschatz why did veteransaffairshawaii keep all the vaccines that were meant to be divided between guam amp cnmi guamcongressman are you aware of this veterans tweeps please retweet,1361543215782879233,2021-02-16,00:09:33,1,NaN,1,6,3
104,yesterday i went with fellow veteran so he could get covid_vaccine at northportvamc never was so impressed in the system they created to get and shots no drama nygovcuomo nycmayor department_of_va Veterans_Benefits_Admin Veterans_Health_Admin,1360577851137929216,2021-02-13,08:13:32,1,1,1,3,11
124,please Veterans_Benefits_Admin department_of_va allow vet spouses to receive the vaccine at the same time the sponsor does multiple trips to different places is inefficient and takes more time presssec vp potus consolidate and crunch the virus,1360307023116664832,2021-02-12,14:17:22,1,NaN,NaN,2,11
178,Veterans_Benefits_Admin department_of_va americas oldest wwii veteran received his covid_vaccine at the veterans affairs medical center in new orleans on monday via wdsu,1359484056325545987,2021-02-10,07:47:11,1,NaN,NaN,6,11
197,Veterans_Benefits_Admin department_of_va marine veteran and world war ii navajo code talker thomas begay survived iwo jima recently he received his covid_vaccine at vanewmexico to protect himself and loved ones from the virus honoringvets coronavirus_pandemicv…,1359197133484744705,2021-02-09,12:47:03,1,NaN,NaN,6,11
301,Veterans_Benefits_Admin department_of_va join bluestarfamily and the redcross virtually on thursday feb at pm et for the militaryconnected families amp the covid_vaccine town hall register to reserve your spot today …,1357022836515344387,2021-02-03,12:47:11,1,NaN,NaN,6,3
309,maziehirono brianschatz department_of_va Veterans_Health_Admin Veterans_Benefits_Admin why did hawaii veterans affairs keep all the vaccines meant to be distributed to all the pacific territories this is unjust amp unfair guamcongressman please retweet,1356793181216493573,2021-02-02,21:34:37,1,NaN,1,6,3
315,josh veteran with nicholas greatdane servicedog got his first coronavirus_pandemicvaccination at the va i hope you are getting your vaccinations too maskup dogs dogtwitter coronavirus_pandemic covid_vaccine vaccine vaccinations Veterans_Benefits_Admin multiplesclerosis,1356690173560512512,2021-02-02,14:45:18,1,NaN,NaN,3,11
316,join bluestarfamily and the redcross virtually on thursday february at pm et for the militaryconnected families amp the covid_vaccine town hall register to reserve your spot today,1356654143893606400,2021-02-02,12:22:07,1,NaN,NaN,6,3


#### Remove Duplicate Tweets Retweet Markers

In [27]:
no_rt_marker = lambda x: re.sub(r"RT @", '',x)

In [190]:
df_tweets_2_c['tweet'] = df_tweets_2_c.tweet.map(no_rt_marker)

In [29]:
df_tweets_2_c.drop_duplicates(subset='tweet',inplace=True)

In [192]:
df_tweets_2_c.shape

(2882, 5)

#### Alpha Numeric and Punctuation Adjustments
Will leave alpha-numeric for first run

In [45]:
alphanumeric = lambda x: re.sub('\w*\d\w*', '', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x.lower())

In [ ]:
# regex = r"(?<!\d)[.,;:](?!\d)"
# test_str = "This is a 1example of the text. But, it only is 2.5 percent of all data"
# remove = re.sub(regex, "", test_str, 0)

In [193]:
df_tweets_2_c['tweet'] = df_tweets_2_c.tweet.map(punc_lower)

#### Remove Websites

In [44]:
no_website = lambda x: re.sub(r"http\S+", '', x)

In [195]:
df_tweets_2_c['tweet'] = df_tweets_2_c.tweet.map(no_website)

#### Remove Other Typical Twitter Elements (If Still Present)

In [388]:
no_at_symbol = lambda x: re.sub(r"@", ' ', x)

In [387]:
no_lb_symbol = lambda x: re.sub(r"#", ' ', x)

In [111]:
remove_department_of_va = lambda x: re.sub(r"departmentofva", '', x)

#### Replace Named Groups

In [488]:
ssa_word_list = ['social security', 'socialsecurity']
ssa_word_list2 = [' ss ',' ssa ']
vba_word_list = ['vavetbenefits','veterans benefits administration', 
                 'veteranbenefitsadministration']
vba_word_list2 = [' vba ']
vha_word_list = ['veteranshealth','veterans health administration']
vha_word_list2 = [' vha ']
veteran_word_list = ['i am a veteran','iamaveteran', 'i am a vet','im a vet']
va_word_list = ['deptvetaffairs']
va_word_list2 = ['the va ']
black_veterans = ['black veterans','black veteran','black vets','blackvets','blackveterans']
covid_list = ['covidpandemic','covid pandemic','covid','coronavirus','covid19','covid-19','covid 19','the pandemic','this pandemic']
vaccine_list = ['coronavirus_pandemicvaccine','covid19vaccine','covidvaccine','covid19 vaccine','covid-19 vaccine','covid19vaccination','covid vaccine','covid19 vaccine',
          'covid-19 vaccines','coronavirus_pandemic vaccine','coronavaccine']


In [357]:
#res = re.sub("|".join(sorted(ssa_word_list, key = len, reverse = True)), repl_wrd, test_str) 

In [358]:
SSA = lambda x: re.sub("|".join(sorted(ssa_word_list, key = len, reverse = True)), 
                       'Social_Security', x)

In [359]:
SSA2 = lambda x: re.sub("|".join(sorted(ssa_word_list2, key = len, reverse = True)), 
                       ' Social_Security ', x)

In [360]:
VBA = lambda x: re.sub("|".join(sorted(vba_word_list, key = len, reverse = True)), 
                       'Veterans_Benefits_Admin', x)

In [361]:
VBA2 = lambda x: re.sub("|".join(sorted(vba_word_list2, key = len, reverse = True)), 
                       ' Veterans_Benefits_Admin ', x)

In [362]:
VHA = lambda x: re.sub("|".join(sorted(vha_word_list, key = len, reverse = True)), 
                       'Veterans_Health_Admin', x)

In [363]:
VHA2 = lambda x: re.sub("|".join(sorted(vha_word_list2, key = len, reverse = True)), 
                       ' Veterans_Health_Admin ', x)

In [364]:
veteran_words = lambda x: re.sub("|".join(sorted(veteran_word_list, key = len, reverse = True)), 
                       'i_am_a_veteran', x)

In [489]:
va_words = lambda x: re.sub("|".join(sorted(va_word_list, key = len, reverse = True)), 
                       'department_of_va', x)

In [490]:
va_words2 = lambda x: re.sub("|".join(sorted(va_word_list2, key = len, reverse = True)), 
                       'department_of_va ', x)

In [367]:
black_veterans_words = lambda x: re.sub("|".join(sorted(black_veterans, key = len, reverse = True)), 
                       'black_veterans', x)

In [423]:
covid_words = lambda x: re.sub("|".join(sorted(covid_list, key = len, reverse = True)), 
                       'coronavirus_pandemic', x)

In [424]:
vaccine_words = lambda x: re.sub("|".join(sorted(vaccine_list, key = len, reverse = True)), 
                       'covid_vaccine', x)

#### Remove Emojis

In [306]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [307]:
remove_emoji('patience🤦‍ Veterans_Benefits_Admin 🙏depar🙏tment_of_va Veterans_Health_Admin')
#emoji("Hi, I am Emoji  😜")

'patience Veterans_Benefits_Admin department_of_va Veterans_Health_Admin'

#### Remove Stop Words

In [40]:
from sklearn.feature_extraction import text 

In [519]:
additional_stop_words = ['rt','vets','vet','veteran','veterans']
additional_stop_words2 = ['rt','vets','vet','veteran','veterans','Veterans_Benefits_Admin','veterans_benefits_admin', 'veterans_health_admin',
                          'Veterans_Health_Admin','department_of_va','veteransbenefitsadmin departmentofva', 
                          'veteransbenefitsadmin departmentofva veterans_health_adminadminadminadminadmin', 
                          'departmentofva veterans_health_adminadminadminadminadmin', 'veteransbenefitsadmin', 
                          'veterans_health_adminadminadminadminadmin', 'departmentofva','va']

stop_wordz = text.ENGLISH_STOP_WORDS.union(additional_stop_words)
stop_wordz2 = text.ENGLISH_STOP_WORDS.union(additional_stop_words2)

## Create Count Vectorizer

In [333]:
text = df_tweets_2_c.tweet

# CountVectorizer is a class; so `vectorizer` below represents an instance of that object.
vectorizer = CountVectorizer(stop_words=stop_wordz)

# call `fit` to build the vocabulary
vectorizer.fit(text)

# then, use `get_feature_names` to return the tokens
vectorizer.get_feature_names()

# finally, call `transform` to convert text to a bag of words
x = vectorizer.transform(text);

In [331]:
print('Sparse Matrix');
# A compressed version; the "sparse" matrix.
#print(type(x))
#print(x)

print ('Matrix')
x_back = x.toarray();
#print(type(x_back))
#print(x_back)

Sparse Matrix
Matrix


In [350]:
df_word_matrix = pd.DataFrame(x_back, columns=vectorizer.get_feature_names())

df_word_matrix.loc["Total"] = df_word_matrix.sum()

df_word_matrix;

## Perform NMF Analysis

#### NMF Tester Function

In [276]:
topic_count = 10
columns_list=[]
for i in range(topic_count):
    columns_list.append('Topic {}'.format(i+1))
print(columns_list)

['Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5', 'Topic 6', 'Topic 7', 'Topic 8', 'Topic 9', 'Topic 10']


In [505]:
def NMF_tester(topic_count,df,doc_word,vectorizer):
    print("TOPIC COUNT {}".format(topic_count))
    nmf_model = NMF(topic_count)
    nmf_doc_topic = nmf_model.fit_transform(doc_word)
    #df_doc_topic = pd.DataFrame(data=doc_topic, columns=["column1", "column2"]) 
    print("Doc Topic Shape is: ", nmf_doc_topic.shape)
    #print(doc_topic[0])
    #print("The Topic matrix is: ",doc_topic)
    print('\n')
    words = vectorizer.get_feature_names()
    t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
    topic_words = [[words[e] for e in l] for l in t]
    print("The Topic Words are: ")
    counter = 1
    for i in topic_words:
        print("Topic {}".format(counter))
        print(i)
        print('-----------------')
        counter+=1
    
    df['topic_count_{}'.format(topic_count)] = nmf_doc_topic.argmax(axis=1)
    print('---------------------------------------------------------------------------')
    columns_list=[]
    for i in range(topic_count):
        columns_list.append('Topic {}'.format(i+1))
    df_doc_topic = pd.DataFrame(nmf_doc_topic, columns=columns_list)
    return df_doc_topic

In [497]:
df_tweets_final.sample(10)

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet
2575,Veterans_Health_Admin the initial sequence of the brain configuration is preprogrammed by the genetic makeup while in the womb,1358314256517758977,2021-02-07,02:18:49,NaN,NaN,1
1129,department_of_va usaa department_of_va va video connect and usaa are all the worst possible options possible few things make my life worse than this trifecta of frustration,1361799518182920195,2021-02-16,17:08:00,NaN,1,NaN
2397,ok chuck mitrecorp solid week w no nose bleeds but seriously you do coach senthomtillis for failure again livermorelab tbi research for Veterans_Health_Admin also goes out to every college football program georgeallenva they cannot eat that,1359281069816762369,2021-02-09,18:20:35,NaN,NaN,1
922,Veterans_Benefits_Admin stimulus payments are automatic for eligible taxpayers including va benefits beneficiaries who didn’t file a tax return irsnews,1346951486073425924,2021-01-06,17:47:14,1,NaN,NaN
2928,addeecee when i started an online degree program with northeastern i contacted their disability services who provided me with a form containing questions i asked my department_of_va Veterans_Health_Admin provider to fill it out he responded department_of_va discourages filling out these forms,1355314389851795462,2021-01-29,19:38:25,NaN,NaN,1
1622,potus Veterans_Health_Admin department_of_va some vas including the martinsburg wv va have not even offered vaccines yet veterans say they waited hours for vaccines at va drivethru event,1361042627257327616,2021-02-14,15:00:23,NaN,1,1
2257,on hold waiting for a rep for an hr with va guy comes on and in the middle of verifying info hangs up come on Veterans_Health_Admin get it together i need my medicine,1359537389962727424,2021-02-10,11:19:07,NaN,NaN,1
1928,secwilkie department_of_va permanently end abusive dog testing sign the petition via change,1360459382727708674,2021-02-13,00:22:47,NaN,1,NaN
907,if you are a bluewaternavy veteran or dependent visit Veterans_Benefits_Admin to find out what resources are available to you,1347212475293429762,2021-01-07,11:04:18,1,NaN,NaN
2241,providing care for disabled veterans is not an easy task va has made a list of caregiving tips for several specific diseases to help you,1359578016658190337,2021-02-10,14:00:33,NaN,NaN,1


In [532]:
nmf_20_topics_doc_topic = NMF_tester(20, df_tweets_final, doc_word_tfidf_ngrams3_sw2,vectorizer_tfidf_ngrams3_sw2)

TOPIC COUNT 20
Doc Topic Shape is:  (2882, 20)


The Topic Words are: 
Topic 1
['uressien', 'pitthealthsci vapittsburgh upmcnews', 'upmcnews', 'vaequity pitthealthsci vapittsburgh', 'vahsrd varesearch pitttweet', 'vapittsburgh']
-----------------
Topic 2
['end abusive dog', 'end abusive', 'permanently end', 'permanently end abusive', 'secwilkie permanently', 'secwilkie permanently end']
-----------------
Topic 3
['vp kamalaharris denismcdonough', 'vp kamalaharris', 'kamalaharris denismcdonough', 'denismcdonough secondgentleman', 'kamalaharris denismcdonough secondgentleman', 'kellyo']
-----------------
Topic 4
['housevetaffairs memphisvamc', 'tormented housevetaffairs', 'disabled years tormented', 'memphisvamc failure cue', 'memphisvamc failure', 'housevetaffairs memphisvamc failure']
-----------------
Topic 5
['militarytimes foxnews', 'usmc johnbrennan stanmcchrystal', 'aarp iava', 'aarp iava teamtrump', 'iava teamtrump veteransunited', 'martindempsey']
-----------------
Topic 6
['vet

In [533]:
nmf_20_topics_doc_topic = NMF_tester(20, df_tweets_final, doc_word_sw_only_cvec,vectorizer_sw_only_cvec)

TOPIC COUNT 20
Doc Topic Shape is:  (2882, 20)


The Topic Words are: 
Topic 1
['uressien', 'vahsrd', 'varesearch', 'pitthealthsci', 'pitttweet', 'vapittsburgh']
-----------------
Topic 2
['service', 'members', 'honor', 'families', 'free', 'thank']
-----------------
Topic 3
['denismcdonough', 'vp', 'secondgentleman', 'kamalaharris', 'kellyo', 'yamiche']
-----------------
Topic 4
['amp', 'support', 'families', 'american', 'capitol', 'doddha']
-----------------
Topic 5
['usarmy', 'usarmyreserve', 'usnavy', 'usmc', 'veteransunited', 'votevets']
-----------------
Topic 6
['care', 'community', 'video', 'usaa', 'live', 'email']
-----------------
Topic 7
['vp', 'years', 'dcvamc', 'good', 'claims', 'congratulations']
-----------------
Topic 8
['help', 'need', 'muellershewrote', 'know', 'needs', 'homeless']
-----------------
Topic 9
['social_security', 'usa', 'votes', 'america', 'fraud', 'trump']
-----------------
Topic 10
['room', 'left', 'waiting', 'services', 'minutes', 'davhq']
------------

In [534]:
nmf_20_topics_doc_topic_cvec_ngrams3 = NMF_tester(20, df_tweets_final, doc_word_sw_only_cvec_ngrams3,vectorizer_sw_only_cvec_ngrams3)

TOPIC COUNT 20
Doc Topic Shape is:  (2882, 20)


The Topic Words are: 
Topic 1
['uressien', 'vahsrd', 'varesearch', 'vahsrd varesearch', 'pitthealthsci', 'pitttweet']
-----------------
Topic 2
['usarmy', 'usarmyreserve', 'usarmy usarmyreserve', 'usnavy', 'veteransunited', 'votevets']
-----------------
Topic 3
['vp', 'dcvamc', 'vp dcvamc', 'years', 'good', 'claims']
-----------------
Topic 4
['vp', 'secondgentleman', 'denismcdonough', 'kamalaharris', 'denismcdonough secondgentleman', 'vp kamalaharris denismcdonough']
-----------------
Topic 5
['learn', 'new', 'information', 'youre', 'resources', 'eligible']
-----------------
Topic 6
['room', 'left', 'waiting', 'waiting room', 'services', 'davhq']
-----------------
Topic 7
['sign', 'end', 'secwilkie', 'dog', 'permanently', 'testing']
-----------------
Topic 8
['amp', 'american', 'support', 'families', 'capitol', 'life']
-----------------
Topic 9
['hhsgov', 'realdonaldtrump', 'fcc', 'cia', 'televisionacad', 'korea']
-----------------
Topi

/Users/arcarter/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [536]:
nmf_20_topics_doc_topic_tfidf_ngrams1 = NMF_tester(20, df_tweets_final, doc_word_tfidf_ngrams1_sw2,vectorizer_tfidf_ngrams1_sw2)

TOPIC COUNT 20
Doc Topic Shape is:  (2882, 20)


The Topic Words are: 
Topic 1
['uressien', 'vahsrd', 'vapittsburgh', 'upmcnews', 'upmc', 'vaequity']
-----------------
Topic 2
['secondgentleman', 'kellyo', 'kamalaharris', 'vp', 'denismcdonough', 'yamiche']
-----------------
Topic 3
['abusive', 'petition', 'testing', 'permanently', 'dog', 'secwilkie']
-----------------
Topic 4
['benefits', 'service', 'learn', 'members', 'disability', 'eligible']
-----------------
Topic 5
['oversight', 'memphisvamc', 'cue', 'tormented', 'sec', 'housevetaffairs']
-----------------
Topic 6
['usarmy', 'usarmyreserve', 'votevets', 'usmc', 'veteransunited', 'usnavy']
-----------------
Topic 7
['muellershewrote', 'denismcdonough', 'howardaesq', 'hugs', 'sorry', 'hang']
-----------------
Topic 8
['vetsweforget', 'veteransfirst', 'peace', 'rest', 'sir', 'barry']
-----------------
Topic 9
['thank', 'sharing', 'peace', 'service', 'sir', 'leoshane']
-----------------
Topic 10
['care', 'health', 'community', 'medica

In [522]:
df_tweets_final.head()

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet,topic_count_10,topic_count_15
0,its time for smccvso sanmateoco veterans to rise up and demand services to be a priority homelessness in our county needs to end smchsa lets put Veterans_Health_Admin and Veterans_Benefits_Admin to work and truly end veteran homelessness in sanmateoco,1361892469500121089,2021-02-16,23:17:22,1,NaN,1,6,1
1,there should be another bill that temporarily raises Social_Security and Veterans_Benefits_Admin to for months new hudgov vouchers for apartments rent rooms amp motel rooms to partially deal w housing crisis expanded snap for unemployed extend lifeline to Social_Security retire bene,1361886795932262407,2021-02-16,22:54:49,1,NaN,NaN,6,8
2,Veterans_Benefits_Admin mr secretary mcdonough good afternoon as you can see from my attached documentation i have been fighting this issue of changing my discharge due to systemic racism for over years,1361866444410155013,2021-02-16,21:33:57,1,NaN,NaN,3,3
3,Veterans_Benefits_Admin in howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in thank you for your …,1361854662488555525,2021-02-16,20:47:08,1,NaN,NaN,6,12
4,Veterans_Benefits_Admin vbatribute trailblazer michelle howard set many firsts during her career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in became the first africanamerican woman to …,1361854659409965057,2021-02-16,20:47:07,1,NaN,NaN,6,8


In [500]:

output = "{:.7f}".format(num)
nmf_10_topics_doc_topic

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10
0,0.000666,1.673880e-02,0.000000e+00,2.631265e-03,0.0,0.000000e+00,0.025972,1.304569e-03,0.005112,0.000000
1,0.000011,7.604345e-05,3.138566e-05,1.762142e-04,0.0,2.060594e-04,0.004293,4.488060e-04,0.000000,0.000000
2,0.000110,2.875220e-05,0.000000e+00,1.172782e-02,0.0,2.303077e-05,0.002162,3.572515e-03,0.000000,0.000000
3,0.000163,5.166710e-05,3.590909e-04,3.262051e-04,0.0,6.212904e-04,0.006995,2.943721e-03,0.000000,0.000000
4,0.000140,7.826969e-05,0.000000e+00,3.810225e-03,0.0,6.307968e-03,0.008437,1.198090e-03,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
2877,0.000000,3.755178e-05,0.000000e+00,2.087480e-04,0.0,1.107868e-07,0.007978,2.188066e-03,0.000000,0.000000
2878,0.000000,5.230360e-08,7.936947e-07,5.713873e-07,0.0,4.776695e-07,0.000028,2.149249e-07,0.000000,0.000000
2879,0.001366,6.322709e-05,0.000000e+00,4.033274e-03,0.0,1.416751e-02,0.004968,2.119047e-03,0.000000,0.000000
2880,0.000000,6.630260e-05,5.306316e-04,3.506890e-04,0.0,3.252871e-05,0.006755,2.307005e-03,0.000028,0.000000


In [520]:
vectorizer_sw_only_cvec = CountVectorizer(stop_words = stop_wordz2)
doc_word_sw_only_cvec = vectorizer_sw_only_cvec.fit_transform(df_tweets_final['tweet'])
doc_word_sw_only_cvec.shape

(2882, 9026)

In [529]:
vectorizer_sw_only_cvec_ngrams3 = CountVectorizer(stop_words = stop_wordz2, ngram_range=(1, 3))
doc_word_sw_only_cvec_ngrams3 = vectorizer_sw_only_cvec_ngrams3.fit_transform(df_tweets_final['tweet'])
doc_word_sw_only_cvec_ngrams3.shape

(2882, 61687)

In [531]:
vectorizer_tfidf_ngrams3_sw2 = TfidfVectorizer(ngram_range=(1,3), stop_words=stop_wordz2)
doc_word_tfidf_ngrams3_sw2 = vectorizer_tfidf_ngrams3_sw2.fit_transform(df_tweets_final['tweet'])
doc_word_tfidf_ngrams3_sw2.shape

(2882, 61687)

In [535]:
vectorizer_tfidf_ngrams1_sw2 = TfidfVectorizer(ngram_range=(1,1), stop_words=stop_wordz2)
doc_word_tfidf_ngrams1_sw2 = vectorizer_tfidf_ngrams1_sw2.fit_transform(df_tweets_final['tweet'])
doc_word_tfidf_ngrams1_sw2.shape

(2882, 9026)

In [514]:
vectorizer_tfidf_ngrams2_sw2 = TfidfVectorizer(ngram_range=(1,2), stop_words=stop_wordz2)
doc_word_tfidf_ngrams2_sw2 = vectorizer_tfidf_ngrams2_sw2.fit_transform(df_tweets_final['tweet'])
doc_word_tfidf_ngrams2_sw2.shape

(2882, 35262)

In [ ]:
doc_word_tfidf_ngrams3_sw2

In [200]:
for i in range(10,11):
    NMF_tester(i, df_tweets_final, doc_word_tfidf_ngrams3_sw2,vectorizer_tfidf_ngrams3_sw2)

TOPIC COUNT 10
Doc Topic Shape is:  (2882, 10)


The Topic Words are: 
['congrats', 'uressiendepartment_of_vasrddepartment_of_vaesearch pitttweet pittgimdepartment_of_vaquity', 'uressiendepartment_of_vasrddepartment_of_vaesearch pitttweet', 'uressiendepartment_of_vasrddepartment_of_vaesearch', 'pitthealthscidepartment_of_vaittsburgh upmcnews upmc', 'upmc']
['secwilkie permanently end', 'permanently end abusive', 'end abusive dog', 'permanently end', 'abusive dog testing', 'dog testing']
['kamalaharris denismcdonough secondgentleman', 'denismcdonough secondgentleman', 'vp kamalaharris', 'vp kamalaharris denismcdonough', 'kamalaharris denismcdonough', 'kellyo']
['vp dcvamc', 'housevetaffairs memphisvamc', 'tormented housevetaffairs', 'oversight claims', 'disabled years tormented', 'years tormented housevetaffairs']
['usarmyreserve', 'usarmy usarmyreserve', 'votevets', 'aarp iava teamtrump', 'foxnews cnnbrk aarp', 'nationalguard usarmy usarmyreserve']
['vetsweforget', 'codeofvets', 'teamm

#### Review Final Topic Data Frame

In [502]:
df_tweets_final.head()

,tweet,id,date,time,VBA Tweet,VA Tweet,VHA Tweet,topic_count_10
0,its time for smccvso sanmateoco veterans to rise up and demand services to be a priority homelessness in our county needs to end smchsa lets put Veterans_Health_Admin and Veterans_Benefits_Admin to work and truly end veteran homelessness in sanmateoco,1361892469500121089,2021-02-16,23:17:22,1,NaN,1,6
1,there should be another bill that temporarily raises Social_Security and Veterans_Benefits_Admin to for months new hudgov vouchers for apartments rent rooms amp motel rooms to partially deal w housing crisis expanded snap for unemployed extend lifeline to Social_Security retire bene,1361886795932262407,2021-02-16,22:54:49,1,NaN,NaN,6
2,Veterans_Benefits_Admin mr secretary mcdonough good afternoon as you can see from my attached documentation i have been fighting this issue of changing my discharge due to systemic racism for over years,1361866444410155013,2021-02-16,21:33:57,1,NaN,NaN,3
3,Veterans_Benefits_Admin in howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in thank you for your …,1361854662488555525,2021-02-16,20:47:08,1,NaN,NaN,6
4,Veterans_Benefits_Admin vbatribute trailblazer michelle howard set many firsts during her career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in became the first africanamerican woman to …,1361854659409965057,2021-02-16,20:47:07,1,NaN,NaN,6


In [180]:
pickle.dump(df_tweets_final, open("df_tweets_final.p", "wb" ) )

In [229]:
df_tweets_final = pickle.load( open( "df_tweets_final.p", "rb" ) )

#### NMF Test with ngrams = 1

In [375]:
for i in range(2,11):
    NMF_tester(i, df_tweets_4_c, doc_word_sw_only_cvec,vectorizer_sw_only_cvec)

TOPIC COUNT 2
Doc Topic Shape is:  (2882, 2)


The Topic Words are: 
['deptvetaffairs', 'veterans', 'veterans_benefits_admin', 'va', 'veterans_health_admin', 'amp']
['veterans_health_admin', 'uressien', 'vahsrd', 'varesearch', 'pitthealthsci', 'pitttweet']
---------------------------------------------------------------------------
TOPIC COUNT 3
Doc Topic Shape is:  (2882, 3)


The Topic Words are: 
['deptvetaffairs', 'veterans_benefits_admin', 'vp', 'denismcdonough', 'veterans_health_admin', 'secondgentleman']
['veterans_health_admin', 'uressien', 'vahsrd', 'varesearch', 'pitthealthsci', 'pitttweet']
['veterans', 'va', 'veterans_benefits_admin', 'veterans_health_admin', 'benefits', 'care']
---------------------------------------------------------------------------
TOPIC COUNT 4
Doc Topic Shape is:  (2882, 4)


The Topic Words are: 
['deptvetaffairs', 'vp', 'denismcdonough', 'veterans_health_admin', 'secondgentleman', 'kamalaharris']
['veterans_health_admin', 'uressien', 'vahsrd', 'vare

#### NMF Test with ngrams = 3

In [376]:
df_tweets_5_c = df_tweets_4_c.copy()

In [386]:
df_tweets_5_c.drop(df_tweets_5_c.iloc[:, 5:14], inplace = True, axis = 1)

In [394]:
vectorizer_sw_only_cvec_ngrams3 = CountVectorizer(stop_words = stop_wordz, ngram_range=(1, 3))
doc_word_sw_only_cvec_ngrams3 = vectorizer_sw_only_cvec_ngrams3.fit_transform(df_tweets_5_c['tweet'])
doc_word_sw_only_cvec_ngrams3.shape

(2882, 66345)

In [395]:
for i in range(2,11):
    NMF_tester(i, df_tweets_5_c, doc_word_sw_only_cvec_ngrams3,vectorizer_sw_only_cvec_ngrams3)

TOPIC COUNT 2
Doc Topic Shape is:  (2882, 2)


The Topic Words are: 
['veterans_health_admin', 'uressien', 'vahsrd', 'varesearch', 'pitthealthsci', 'vahsrd varesearch']
['deptvetaffairs', 'veterans_benefits_admin', 'veterans', 'va', 'veterans_health_admin', 'amp']
---------------------------------------------------------------------------
TOPIC COUNT 3
Doc Topic Shape is:  (2882, 3)


The Topic Words are: 
['veterans_health_admin', 'uressien', 'vahsrd', 'varesearch', 'vahsrd varesearch', 'pitthealthsci']
['veterans', 'va', 'veterans_benefits_admin', 'veterans_health_admin', 'amp', 'care']
['deptvetaffairs', 'vp', 'veterans_benefits_admin', 'denismcdonough', 'deptvetaffairs denismcdonough', 'deptvetaffairs veterans_benefits_admin']
---------------------------------------------------------------------------
TOPIC COUNT 4
Doc Topic Shape is:  (2882, 4)


The Topic Words are: 
['veterans_health_admin', 'uressien', 'vahsrd', 'varesearch', 'vahsrd varesearch', 'pitthealthsci']
['veterans', '

#### Create TF-IDF Vectorizer

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### NMF with TF-IDF Ngram1

In [401]:
vectorizer_tfidf_ngrams1 = TfidfVectorizer(ngram_range=(1,1), stop_words=stop_wordz)
doc_word_tfidf_ngrams1 = vectorizer_tfidf_ngrams1.fit_transform(df_tweets_7_c['tweet'])
doc_word_tfidf_ngrams1.shape

(2882, 9042)

In [402]:
for i in range(2,11):
    NMF_tester(i, df_tweets_7_c, doc_word_tfidf_ngrams1,vectorizer_tfidf_ngrams1)

TOPIC COUNT 2
Doc Topic Shape is:  (2882, 2)


The Topic Words are: 
['uressien', 'vahsrd', 'upmc', 'vapittsburgh', 'upmcnews', 'pittgim']
['deptvetaffairs', 'veterans_benefits_admin', 'veterans', 'veterans_health_admin', 'denismcdonough', 'vetsweforget']
---------------------------------------------------------------------------
TOPIC COUNT 3
Doc Topic Shape is:  (2882, 3)


The Topic Words are: 
['uressien', 'vahsrd', 'upmcnews', 'vapittsburgh', 'upmc', 'pittgim']
['veterans_benefits_admin', 'deptvetaffairs', 'veterans', 'veterans_health_admin', 'vetsweforget', 'va']
['denismcdonough', 'vp', 'secondgentleman', 'deptvetaffairs', 'kellyo', 'kamalaharris']
---------------------------------------------------------------------------
TOPIC COUNT 4
Doc Topic Shape is:  (2882, 4)


The Topic Words are: 
['uressien', 'vahsrd', 'upmc', 'vaequity', 'vapittsburgh', 'upmcnews']
['veterans_benefits_admin', 'deptvetaffairs', 'veterans', 'veterans_health_admin', 'vetsweforget', 'va']
['denismcdonoug

#### NMF with TF-IDF Ngram3

In [ ]:
vectorizer_tfidf_ngrams3 = TfidfVectorizer(ngram_range=(1,3), stop_words=stop_wordz2)
doc_word_tfidf = vectorizer_tfidf_ngrams3.fit_transform(df_tweets_6_c['tweet'])
doc_word_tfidf.shape

In [397]:
for i in range(2,11):
    NMF_tester(i, df_tweets_6_c, doc_word_tfidf,vectorizer_tfidf_ngrams3)

TOPIC COUNT 2
Doc Topic Shape is:  (2882, 2)


The Topic Words are: 
['uressien', 'vapittsburgh upmcnews', 'upmc veterans_health_admin', 'vaequity pitthealthsci vapittsburgh', 'pitthealthsci vapittsburgh', 'vaequity pitthealthsci']
['veterans_benefits_admin deptvetaffairs', 'veterans_benefits_admin deptvetaffairs veterans_health_admin', 'deptvetaffairs', 'veterans_benefits_admin', 'deptvetaffairs veterans_health_admin', 'veterans_health_admin']
---------------------------------------------------------------------------
TOPIC COUNT 3
Doc Topic Shape is:  (2882, 3)


The Topic Words are: 
['uressien', 'vapittsburgh upmcnews upmc', 'vaequity pitthealthsci', 'pitthealthsci vapittsburgh upmcnews', 'pitthealthsci vapittsburgh', 'vaequity']
['veterans_benefits_admin deptvetaffairs', 'veterans_benefits_admin deptvetaffairs veterans_health_admin', 'veterans_benefits_admin', 'deptvetaffairs veterans_health_admin', 'deptvetaffairs', 'veterans_health_admin']
['deptvetaffairs denismcdonough', 'deni

Doc Topic Shape is:  (2882, 10)


The Topic Words are: 
['uressien', 'varesearch pitttweet', 'vaequity', 'vapittsburgh upmcnews upmc', 'vaequity pitthealthsci', 'vapittsburgh upmcnews']
['veterans_benefits_admin deptvetaffairs veterans_health_admin', 'veterans_benefits_admin deptvetaffairs', 'deptvetaffairs veterans_health_admin', 'veterans veterans_benefits_admin deptvetaffairs', 'veterans veterans_benefits_admin', 'deptvetaffairs']
['kamalaharris deptvetaffairs denismcdonough', 'vp kamalaharris', 'vp kamalaharris deptvetaffairs', 'denismcdonough secondgentleman', 'deptvetaffairs denismcdonough secondgentleman', 'kamalaharris deptvetaffairs']
['deptvetaffairs permanently', 'permanently end abusive', 'abusive dog testing', 'secwilkie deptvetaffairs permanently', 'end abusive dog', 'dog testing']
['vp dcvamc', 'veterans_benefits_admin housevetaffairs memphisvamc', 'deptvetaffairs veterans_benefits_admin housevetaffairs', 'housevetaffairs memphisvamc', 'veterans_benefits_admin housevetaf

/Users/arcarter/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


### Summary of Vectorizers

In [515]:
vectorizer_sw_only_cvec = CountVectorizer(stop_words = stop_wordz)
doc_word_sw_only_cvec = vectorizer_sw_only_cvec.fit_transform(df_tweets_final['tweet'])
doc_word_sw_only_cvec.shape

(2882, 9030)

In [516]:
vectorizer_sw_only_cvec_ngrams3 = CountVectorizer(stop_words = stop_wordz, ngram_range=(1, 3))
doc_word_sw_only_cvec_ngrams3 = vectorizer_sw_only_cvec_ngrams3.fit_transform(df_tweets_final['tweet'])
doc_word_sw_only_cvec_ngrams3.shape

(2882, 65500)

In [ ]:
vectorizer_tfidf_ngrams1 = TfidfVectorizer(ngram_range=(1,1), stop_words=stop_wordz)
doc_word_tfidf_ngrams1 = vectorizer_tfidf_ngrams1.fit_transform(df_tweets_7_c['tweet'])
doc_word_tfidf_ngrams1.shape

In [429]:
vectorizer_tfidf_ngrams3 = TfidfVectorizer(ngram_range=(1,3), stop_words=stop_wordz)
doc_word_tfidf = vectorizer_tfidf_ngrams3.fit_transform(df_tweets_6_c['tweet'])
doc_word_tfidf.shape

(2882, 66345)

### Perform Latent Semantic Analysis (LSA)

In [404]:
from sklearn.decomposition import TruncatedSVD

#### LSA with CVEC

In [405]:
lsa = TruncatedSVD(2)
doc_topic = lsa.fit_transform(doc_word_sw_only_cvec)
lsa.explained_variance_ratio_

array([0.01165783, 0.04053339])

In [408]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2"],
             columns = vectorizer_sw_only_cvec.get_feature_names())
topic_word;

In [409]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [411]:
display_topics(lsa, vectorizer_sw_only_cvec.get_feature_names(), 10)


Topic  0
deptvetaffairs, veterans, veterans_benefits_admin, veterans_health_admin, va, amp, care, uressien, veteran, varesearch

Topic  1
veterans_health_admin, uressien, vahsrd, varesearch, pitthealthsci, upmc, upmcnews, vapittsburgh, vaequity, pittgim


#### LSA Helper Function

In [422]:
def lsa_model(topic_count,doc_word_vector,vectorizer,num_words):
    print('This is Topic Count {}'.format(topic_count))
    lsa = TruncatedSVD(topic_count)
    doc_topic = lsa.fit_transform(doc_word_vector)
    feature_names_lsa = vectorizer.get_feature_names()
    display_topics(lsa, vectorizer.get_feature_names(), num_words)
    print('---------------------------------------------------------------------------')
    return None

In [423]:
lsa_model(5,doc_word_sw_only_cvec,vectorizer_sw_only_cvec,8)

This is Topic Count 5

Topic  0
deptvetaffairs, veterans, veterans_benefits_admin, veterans_health_admin, va, amp, care, uressien

Topic  1
veterans_health_admin, uressien, vahsrd, varesearch, pitthealthsci, upmc, upmcnews, vapittsburgh

Topic  2
deptvetaffairs, vp, denismcdonough, secondgentleman, kamalaharris, kellyo, muellershewrote, dcvamc

Topic  3
veterans_benefits_admin, social_security, usa, benefits, usnavy, votes, uressien, vaequity

Topic  4
veterans_health_admin, veterans, amp, brain, blood, black, today, heart
---------------------------------------------------------------------------


In [425]:
lsa_model(5,doc_word_tfidf_ngrams1,vectorizer_tfidf_ngrams1,8)

This is Topic Count 5

Topic  0
uressien, vahsrd, upmcnews, upmc, vapittsburgh, vaequity, pittgim, pitttweet

Topic  1
deptvetaffairs, veterans_benefits_admin, veterans, veterans_health_admin, denismcdonough, vetsweforget, va, vp

Topic  2
denismcdonough, vp, secondgentleman, kellyo, kamalaharris, deptvetaffairs, muellershewrote, yamiche

Topic  3
abusive, petition, testing, permanently, dog, secwilkie, end, sign

Topic  4
va, care, benefits, health, learn, veteran, veterans, help
---------------------------------------------------------------------------


In [427]:
lsa_model(5,doc_word_sw_only_cvec_ngrams3,vectorizer_sw_only_cvec_ngrams3,8)

This is Topic Count 5

Topic  0
veterans_health_admin, uressien, vahsrd, varesearch, pitthealthsci, vahsrd varesearch, pitttweet, vaequity pitthealthsci

Topic  1
deptvetaffairs, veterans_benefits_admin, veterans, va, veterans_health_admin, amp, veterans_benefits_admin deptvetaffairs, vp

Topic  2
deptvetaffairs, vp, denismcdonough, deptvetaffairs denismcdonough, secondgentleman, kamalaharris, kamalaharris deptvetaffairs, denismcdonough secondgentleman

Topic  3
veterans_benefits_admin, deptvetaffairs veterans_benefits_admin, usnavy, usarmy, usmc, iava, veteransunited, fbi

Topic  4
deptvetaffairs, denismcdonough, usarmy, deptvetaffairs denismcdonough, usmc, usarmyreserve, usarmy usarmyreserve, militarytimes
---------------------------------------------------------------------------


In [431]:
lsa_model(5,doc_word_tfidf,vectorizer_tfidf_ngrams3,8)

This is Topic Count 5

Topic  0
uressien, vapittsburgh upmcnews upmc, pitthealthsci vapittsburgh, pittgim, vahsrd varesearch pitttweet, pitthealthsci vapittsburgh upmcnews, upmc veterans_health_admin, vaequity pitthealthsci

Topic  1
veterans_benefits_admin deptvetaffairs, veterans_benefits_admin deptvetaffairs veterans_health_admin, deptvetaffairs, veterans_benefits_admin, deptvetaffairs veterans_health_admin, veterans_health_admin, veterans veterans_benefits_admin deptvetaffairs, veterans veterans_benefits_admin

Topic  2
deptvetaffairs denismcdonough, denismcdonough, secondgentleman, kamalaharris deptvetaffairs denismcdonough, vp kamalaharris deptvetaffairs, vp kamalaharris, denismcdonough secondgentleman, deptvetaffairs denismcdonough secondgentleman

Topic  3
dog testing, abusive dog testing, secwilkie deptvetaffairs permanently, deptvetaffairs permanently end, deptvetaffairs permanently, end abusive, end abusive dog, permanently end abusive

Topic  4
vp dcvamc, deptvetaffairs vet

In [434]:
lsa_model(5,doc_word_tfidf,vectorizer_tfidf_ngrams3,8)

This is Topic Count 5

Topic  0
uressien, vapittsburgh upmcnews upmc, pitthealthsci vapittsburgh, pittgim, vahsrd varesearch pitttweet, pitthealthsci vapittsburgh upmcnews, upmc veterans_health_admin, vaequity pitthealthsci

Topic  1
veterans_benefits_admin deptvetaffairs, veterans_benefits_admin deptvetaffairs veterans_health_admin, deptvetaffairs, veterans_benefits_admin, deptvetaffairs veterans_health_admin, veterans_health_admin, veterans veterans_benefits_admin deptvetaffairs, veterans veterans_benefits_admin

Topic  2
deptvetaffairs denismcdonough, denismcdonough, secondgentleman, deptvetaffairs denismcdonough secondgentleman, vp kamalaharris deptvetaffairs, kamalaharris deptvetaffairs denismcdonough, vp kamalaharris, denismcdonough secondgentleman

Topic  3
end abusive, deptvetaffairs permanently end, abusive dog testing, end abusive dog, secwilkie deptvetaffairs permanently, permanently end, permanently end abusive, dog testing

Topic  4
vp dcvamc, housevetaffairs memphisvamc, 

___
## Early Sentiment Analysis

In [256]:
from textblob import TextBlob

In [257]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [326]:
sid_obj = SentimentIntensityAnalyzer()
sentiment = []
for text in df_tweets_3_c.tweet:
    sentiment.append(sid_obj.polarity_scores(text))

In [327]:
sentiment

[{'neg': 0.038, 'neu': 0.888, 'pos': 0.074, 'compound': 0.34},
 {'neg': 0.094, 'neu': 0.868, 'pos': 0.039, 'compound': -0.5267},
 {'neg': 0.176, 'neu': 0.747, 'pos': 0.077, 'compound': -0.5719},
 {'neg': 0.0, 'neu': 0.815, 'pos': 0.185, 'compound': 0.8126},
 {'neg': 0.098, 'neu': 0.902, 'pos': 0.0, 'compound': -0.5994},
 {'neg': 0.0, 'neu': 0.819, 'pos': 0.181, 'compound': 0.8126},
 {'neg': 0.087, 'neu': 0.913, 'pos': 0.0, 'compound': -0.5994},
 {'neg': 0.073, 'neu': 0.789, 'pos': 0.138, 'compound': 0.4215},
 {'neg': 0.136, 'neu': 0.864, 'pos': 0.0, 'compound': -0.5267},
 {'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'compound': 0.714},
 {'neg': 0.164, 'neu': 0.567, 'pos': 0.269, 'compound': 0.4939},
 {'neg': 0.0, 'neu': 0.929, 'pos': 0.071, 'compound': 0.3182},
 {'neg': 0.0, 'neu': 0.867, 'pos': 0.133, 'compound': 0.6369},
 {'neg': 0.0, 'neu': 0.871, 'pos': 0.129, 'compound': 0.6369},
 {'neg': 0.217, 'neu': 0.783, 'pos': 0.0, 'compound': -0.7269},
 {'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 

In [328]:
pd.concat([df_tweets_3_c,pd.DataFrame(sentiment)], axis=1)

,tweet,time,VBA Tweet,VA Tweet,VHA Tweet,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,neg,neu,pos,compound
0,its time for smccvso sanmateoco veterans to rise up and demand services to be a priority homelessness in our county needs to endsmchsa lets put Veterans_Health_Admin and Veterans_Benefits_Admin to work and truly end veteran homelessness in sanmateoco,23:17:22,1,NaN,1,0.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,0.038,0.888,0.074,0.3400
1,there should be another bill that temporarily raises Social_Security and Veterans_Benefits_Admin to for months new hudgov vouchers for apartments rent rooms amp motel rooms to partially deal w housing crisis expanded snap for unemployed extend lifeline to Social_Security retire bene,22:54:49,1,NaN,NaN,0.0,2.0,3.0,3.0,3.0,6.0,6.0,6.0,6.0,0.094,0.868,0.039,-0.5267
2,Veterans_Benefits_Admin mr secretary mcdonough good afternoon as you can see from my attached documentation i have been fighting this issue of changing my discharge due to systemic racism for over years,21:33:57,1,NaN,NaN,0.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.176,0.747,0.077,-0.5719
3,Veterans_Benefits_Admin in howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in thank you for your …,20:47:08,1,NaN,NaN,0.0,2.0,3.0,3.0,3.0,6.0,6.0,6.0,6.0,0.000,0.815,0.185,0.8126
4,Veterans_Benefits_Admin vbatribute trailblazer michelle howard set many firsts during her career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in became the first africanamerican woman to …,20:47:07,1,NaN,NaN,0.0,2.0,3.0,3.0,3.0,6.0,6.0,6.0,6.0,0.098,0.902,0.000,-0.5994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2980,va clinicians are using annie to help patients easily monitor and share their health readings find out more,15:03:06,NaN,NaN,1,0.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN
2981,frannicosia hsratbmc Veterans_Health_Admin varesearch snuggling 🥰,14:36:54,NaN,NaN,1,1.0,1.0,1.0,1.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN
2982,hay mtgarena usnavy deptvetaffairs Veterans_Health_Admin airforceassoc usarmy usairforce try this 😎 amazing past time for veteran metal health and well twitch or other youtube,14:16:33,NaN,NaN,1,0.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN
2983,if you are choosing between paying rent or buying groceries va can help call for housing assistance if you don’t have a phone or internet visit your closest va medical center for more info visit,14:00:12,NaN,NaN,1,0.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN


In [217]:
doc_topic;

In [218]:
doc_topic.argmax(axis=1);

In [219]:
df_tweets_2_c['topic2'] = doc_topic.argmax(axis=1)

In [222]:
df_tweets_2_c.head()

,tweet,time,VBA Tweet,VA Tweet,VHA Tweet,topic2
0,its time for smccvso sanmateoco veterans to rise up and demand services to be a priority homelessness in our county needs to endsmchsa lets put Veterans_Health_Admin and Veterans_Benefits_Admin to work and truly end veteran homelessness in sanmateoco,23:17:22,1,NaN,1,0
1,there should be another bill that temporarily raises Social_Security and Veterans_Benefits_Admin to 2500 for 612 months new hudgov vouchers for apartments rent rooms amp motel rooms to partially deal w housing crisis expanded snap for unemployed extend lifeline toSocial_Securityretire bene,22:54:49,1,NaN,NaN,0
2,Veterans_Benefits_Admin mr secretary mcdonough good afternoon as you can see from my attached documentation i have been fighting this issue of changing my discharge due to systemic racism for over 2 years,21:33:57,1,NaN,NaN,0
3,Veterans_Benefits_Admin 22 in 2014 howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in 2017 thank you for your …,20:47:08,1,NaN,NaN,0
4,Veterans_Benefits_Admin vbatribute trailblazer michelle howard set many firsts during her 35year career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in 1999 became the first africanamerican woman to …,20:47:07,1,NaN,NaN,0


### NMF(3)

In [223]:
nmf_model = NMF(3)
doc_topic = nmf_model.fit_transform(doc_word)
doc_topic.shape

(2882, 3)

In [224]:
words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

[['deptvetaffairs',
  'veterans_benefits_admin',
  'vp',
  'denismcdonough',
  'veterans_health_admin',
  'secondgentleman'],
 ['veterans_health_admin',
  'uressien',
  'vahsrd',
  'varesearch',
  'pitthealthsci',
  'pitttweet'],
 ['veterans',
  'va',
  'veterans_benefits_admin',
  'care',
  'benefits',
  'veterans_health_admin']]

* Topic 1 seems to be about ..
* Topic 2 ...
* Topic 3 ...

In [225]:
doc_topic;

In [226]:
doc_topic.argmax(axis=1);

In [228]:
df_tweets_2_c['topic3'] = doc_topic.argmax(axis=1)

In [229]:
df_tweets_2_c.head()

,tweet,time,VBA Tweet,VA Tweet,VHA Tweet,topic2,topic3
0,its time for smccvso sanmateoco veterans to rise up and demand services to be a priority homelessness in our county needs to endsmchsa lets put Veterans_Health_Admin and Veterans_Benefits_Admin to work and truly end veteran homelessness in sanmateoco,23:17:22,1,NaN,1,0,2
1,there should be another bill that temporarily raises Social_Security and Veterans_Benefits_Admin to 2500 for 612 months new hudgov vouchers for apartments rent rooms amp motel rooms to partially deal w housing crisis expanded snap for unemployed extend lifeline toSocial_Securityretire bene,22:54:49,1,NaN,NaN,0,2
2,Veterans_Benefits_Admin mr secretary mcdonough good afternoon as you can see from my attached documentation i have been fighting this issue of changing my discharge due to systemic racism for over 2 years,21:33:57,1,NaN,NaN,0,2
3,Veterans_Benefits_Admin 22 in 2014 howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in 2017 thank you for your …,20:47:08,1,NaN,NaN,0,2
4,Veterans_Benefits_Admin vbatribute trailblazer michelle howard set many firsts during her 35year career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in 1999 became the first africanamerican woman to …,20:47:07,1,NaN,NaN,0,2


### NMF(4)

In [237]:
nmf_model = NMF(4)
doc_topic = nmf_model.fit_transform(doc_word)
doc_topic.shape

(2882, 4)

In [238]:
words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

[['deptvetaffairs',
  'vp',
  'denismcdonough',
  'veterans_health_admin',
  'secondgentleman',
  'kamalaharris'],
 ['veterans_health_admin',
  'uressien',
  'vahsrd',
  'varesearch',
  'pitthealthsci',
  'pitttweet'],
 ['veterans', 'va', 'care', 'veterans_health_admin', 'health', 'amp'],
 ['veterans_benefits_admin',
  'social_security',
  'benefits',
  'usa',
  'veteran',
  'usnavy']]

* Topic 1 seems to be about ..
* Topic 2 ...
* Topic 3 ...

In [239]:
doc_topic;

In [240]:
doc_topic.argmax(axis=1);

In [241]:
df_tweets_2_c['topic4'] = doc_topic.argmax(axis=1)

In [242]:
df_tweets_2_c.head()

,tweet,time,VBA Tweet,VA Tweet,VHA Tweet,topic2,topic3,topic4
0,its time for smccvso sanmateoco veterans to rise up and demand services to be a priority homelessness in our county needs to endsmchsa lets put Veterans_Health_Admin and Veterans_Benefits_Admin to work and truly end veteran homelessness in sanmateoco,23:17:22,1,NaN,1,0,2,3
1,there should be another bill that temporarily raises Social_Security and Veterans_Benefits_Admin to 2500 for 612 months new hudgov vouchers for apartments rent rooms amp motel rooms to partially deal w housing crisis expanded snap for unemployed extend lifeline toSocial_Securityretire bene,22:54:49,1,NaN,NaN,0,2,3
2,Veterans_Benefits_Admin mr secretary mcdonough good afternoon as you can see from my attached documentation i have been fighting this issue of changing my discharge due to systemic racism for over 2 years,21:33:57,1,NaN,NaN,0,2,3
3,Veterans_Benefits_Admin 22 in 2014 howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in 2017 thank you for your …,20:47:08,1,NaN,NaN,0,2,3
4,Veterans_Benefits_Admin vbatribute trailblazer michelle howard set many firsts during her 35year career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in 1999 became the first africanamerican woman to …,20:47:07,1,NaN,NaN,0,2,3


### NMF(5)

In [244]:
nmf_model = NMF(5)
doc_topic = nmf_model.fit_transform(doc_word)
doc_topic.shape

(2882, 5)

In [245]:
words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

[['deptvetaffairs',
  'vp',
  'denismcdonough',
  'veterans_health_admin',
  'secondgentleman',
  'kamalaharris'],
 ['veterans_health_admin',
  'uressien',
  'vahsrd',
  'varesearch',
  'pitthealthsci',
  'pitttweet'],
 ['veterans', 'veterans_health_admin', 'amp', 'care', 'service', 'black'],
 ['veterans_benefits_admin',
  'social_security',
  'benefits',
  'usa',
  'veteran',
  'usnavy'],
 ['va', 'health', 'care', 'benefits', 'help', 'learn']]

* Topic 1 seems to be about ..
* Topic 2 ...
* Topic 3 ...

In [246]:
doc_topic;

In [247]:
doc_topic.argmax(axis=1);

In [248]:
df_tweets_2_c['topic5'] = doc_topic.argmax(axis=1)

In [369]:
df_tweets_4_c.head()

,tweet,time,VBA Tweet,VA Tweet,VHA Tweet,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10
0,its time for smccvso sanmateoco veterans to rise up and demand services to be a priority homelessness in our county needs to endsmchsa lets put Veterans_Health_Admin and Veterans_Benefits_Admin to work and truly end veteran homelessness in sanmateoco,23:17:22,1,NaN,1,0,2,3,4,2,2,2,2,2
1,there should be another bill that temporarily raises Social_Security and Veterans_Benefits_Admin to for months new hudgov vouchers for apartments rent rooms amp motel rooms to partially deal w housing crisis expanded snap for unemployed extend lifeline to Social_Security retire bene,22:54:49,1,NaN,NaN,0,2,3,3,3,3,3,3,3
2,Veterans_Benefits_Admin mr secretary mcdonough good afternoon as you can see from my attached documentation i have been fighting this issue of changing my discharge due to systemic racism for over years,21:33:57,1,NaN,NaN,0,2,3,3,3,3,3,3,3
3,Veterans_Benefits_Admin in howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in thank you for your …,20:47:08,1,NaN,NaN,0,2,3,3,3,6,6,6,6
4,Veterans_Benefits_Admin vbatribute trailblazer michelle howard set many firsts during her career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in became the first africanamerican woman to …,20:47:07,1,NaN,NaN,0,2,3,3,3,6,6,6,6


### NMF Tester Function

In [318]:
def NMF_tester(topic_count,df):
    print("TOPIC COUNT {}".format(topic_count))
    nmf_model = NMF(topic_count)
    doc_topic = nmf_model.fit_transform(doc_word)
    print("Doc Topic Shape is: ", doc_topic.shape)
    #print("The Topic matrix is: ",doc_topic)
    print('\n')
    words = vectorizer.get_feature_names()
    t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
    topic_words = [[words[e] for e in l] for l in t]
    print("The Topic Words are: ")
    for i in topic_words:
        print(i)
    
    df['topic{}'.format(topic_count)] = doc_topic.argmax(axis=1)
    print('---------------------------------------------------------------------------')
    return None

In [319]:
NMF_tester(9,df_tweets_2_c)

TOPIC COUNT 9
Doc Topic Shape is:  (2882, 9)


The Topic Words are: 
['deptvetaffairs', 'muellershewrote', 'denismcdonough', 'just', 'need', 'thank']
['uressien', 'vahsrd', 'varesearch', 'pitthealthsci', 'pitttweet', 'vaequity']
['veterans', 'service', 'black', 'care', 'services', 'benefits']
['veterans_benefits_admin', 'social_security', 'benefits', 'usa', 'veteran', 'votes']
['va', 'health', 'care', 'benefits', 'help', 'learn']
['veterans_health_admin', 'care', 'vaccine', 'brain', 'blood', 'heart']
['amp', 'service', 'veteran', 'today', 'support', 'families']
['vp', 'denismcdonough', 'deptvetaffairs', 'secondgentleman', 'kamalaharris', 'kellyo']
['usarmy', 'usmc', 'usarmyreserve', 'usnavy', 'militarytimes', 'iava']
---------------------------------------------------------------------------


In [325]:
for i in range(2,11):
    NMF_tester(i,df_tweets_3_c)

TOPIC COUNT 2
Doc Topic Shape is:  (2882, 2)


The Topic Words are: 
['deptvetaffairs', 'veterans', 'veterans_benefits_admin', 'va', 'veterans_health_admin', 'amp']
['veterans_health_admin', 'uressien', 'vahsrd', 'varesearch', 'pitthealthsci', 'pitttweet']
---------------------------------------------------------------------------
TOPIC COUNT 3
Doc Topic Shape is:  (2882, 3)


The Topic Words are: 
['deptvetaffairs', 'veterans_benefits_admin', 'vp', 'denismcdonough', 'veterans_health_admin', 'secondgentleman']
['veterans_health_admin', 'uressien', 'vahsrd', 'varesearch', 'pitthealthsci', 'pitttweet']
['veterans', 'va', 'veterans_benefits_admin', 'care', 'benefits', 'veterans_health_admin']
---------------------------------------------------------------------------
TOPIC COUNT 4
Doc Topic Shape is:  (2882, 4)


The Topic Words are: 
['deptvetaffairs', 'vp', 'denismcdonough', 'veterans_health_admin', 'secondgentleman', 'kamalaharris']
['veterans_health_admin', 'uressien', 'vahsrd', 'vare

___
## Early Sentiment Analysis

In [256]:
from textblob import TextBlob

In [257]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [326]:
sid_obj = SentimentIntensityAnalyzer()
sentiment = []
for text in df_tweets_3_c.tweet:
    sentiment.append(sid_obj.polarity_scores(text))

In [327]:
sentiment

[{'neg': 0.038, 'neu': 0.888, 'pos': 0.074, 'compound': 0.34},
 {'neg': 0.094, 'neu': 0.868, 'pos': 0.039, 'compound': -0.5267},
 {'neg': 0.176, 'neu': 0.747, 'pos': 0.077, 'compound': -0.5719},
 {'neg': 0.0, 'neu': 0.815, 'pos': 0.185, 'compound': 0.8126},
 {'neg': 0.098, 'neu': 0.902, 'pos': 0.0, 'compound': -0.5994},
 {'neg': 0.0, 'neu': 0.819, 'pos': 0.181, 'compound': 0.8126},
 {'neg': 0.087, 'neu': 0.913, 'pos': 0.0, 'compound': -0.5994},
 {'neg': 0.073, 'neu': 0.789, 'pos': 0.138, 'compound': 0.4215},
 {'neg': 0.136, 'neu': 0.864, 'pos': 0.0, 'compound': -0.5267},
 {'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'compound': 0.714},
 {'neg': 0.164, 'neu': 0.567, 'pos': 0.269, 'compound': 0.4939},
 {'neg': 0.0, 'neu': 0.929, 'pos': 0.071, 'compound': 0.3182},
 {'neg': 0.0, 'neu': 0.867, 'pos': 0.133, 'compound': 0.6369},
 {'neg': 0.0, 'neu': 0.871, 'pos': 0.129, 'compound': 0.6369},
 {'neg': 0.217, 'neu': 0.783, 'pos': 0.0, 'compound': -0.7269},
 {'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 

In [328]:
pd.concat([df_tweets_3_c,pd.DataFrame(sentiment)], axis=1)

,tweet,time,VBA Tweet,VA Tweet,VHA Tweet,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,neg,neu,pos,compound
0,its time for smccvso sanmateoco veterans to rise up and demand services to be a priority homelessness in our county needs to endsmchsa lets put Veterans_Health_Admin and Veterans_Benefits_Admin to work and truly end veteran homelessness in sanmateoco,23:17:22,1,NaN,1,0.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,0.038,0.888,0.074,0.3400
1,there should be another bill that temporarily raises Social_Security and Veterans_Benefits_Admin to for months new hudgov vouchers for apartments rent rooms amp motel rooms to partially deal w housing crisis expanded snap for unemployed extend lifeline to Social_Security retire bene,22:54:49,1,NaN,NaN,0.0,2.0,3.0,3.0,3.0,6.0,6.0,6.0,6.0,0.094,0.868,0.039,-0.5267
2,Veterans_Benefits_Admin mr secretary mcdonough good afternoon as you can see from my attached documentation i have been fighting this issue of changing my discharge due to systemic racism for over years,21:33:57,1,NaN,NaN,0.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.176,0.747,0.077,-0.5719
3,Veterans_Benefits_Admin in howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in thank you for your …,20:47:08,1,NaN,NaN,0.0,2.0,3.0,3.0,3.0,6.0,6.0,6.0,6.0,0.000,0.815,0.185,0.8126
4,Veterans_Benefits_Admin vbatribute trailblazer michelle howard set many firsts during her career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in became the first africanamerican woman to …,20:47:07,1,NaN,NaN,0.0,2.0,3.0,3.0,3.0,6.0,6.0,6.0,6.0,0.098,0.902,0.000,-0.5994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2980,va clinicians are using annie to help patients easily monitor and share their health readings find out more,15:03:06,NaN,NaN,1,0.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN
2981,frannicosia hsratbmc Veterans_Health_Admin varesearch snuggling 🥰,14:36:54,NaN,NaN,1,1.0,1.0,1.0,1.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN
2982,hay mtgarena usnavy deptvetaffairs Veterans_Health_Admin airforceassoc usarmy usairforce try this 😎 amazing past time for veteran metal health and well twitch or other youtube,14:16:33,NaN,NaN,1,0.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN
2983,if you are choosing between paying rent or buying groceries va can help call for housing assistance if you don’t have a phone or internet visit your closest va medical center for more info visit,14:00:12,NaN,NaN,1,0.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN


## Create TF-IDF for Further Processing